```
Copyright 2023 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Day in Riverbend Example

An illustrative social simulation with 5 players which simulates a normal day in an imaginary town caller Riverbend. Each player has their own configurable backstory. The agents are configured to re-implement the architecure in Park et al. (2023) - they have reflection, plan, and identity components; their associative memory uses importance function. This is _not_ an exact re-implementation.

Park, J.S., O'Brien, J.C., Cai, C.J., Morris, M.R., Liang, P. and Bernstein, M.S., 2023. Generative agents: Interactive simulacra of human behavior. arXiv preprint arXiv:2304.03442.

## Init and import

In [ ]:
# @title Imports

import concurrent.futures
import datetime
import random

from google.colab import widgets
from IPython import display

from concordia.agents import basic_agent
from concordia.agents import components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import embedder_st5
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.environment import components as gm_components
from concordia.environment import game_master
from concordia.environment.metrics import goal_achievement
from concordia.language_model import sax_model
from concordia.utils import html as html_lib
from concordia.utils import plotting

In [ ]:
# @title Setup sentence encoder
embedder = embedder_st5.EmbedderST5()

In [ ]:
# @title SAX Language Model

# Add path to your SAX server here:
SAX_PATH = '' # @param {type:"string"}
DEFAULT_MAX_TOKENS = 300  # @param {type: 'integer'}
DEFAULT_TIMEOUT_SECONDS = 60  # @param {type: 'number'}

model = sax_model.SAXLanguageModel(SAX_PATH)

## Configuring the generic knowledge of players and GM.

In [ ]:
#@title Make importance models

importance_model = importance_function.AgentImportanceModel(model)
importance_model_gm = importance_function.ConstantImportanceModel()


In [ ]:
#@title Make the clock
SETUP_TIME = datetime.datetime(hour=8, year=2024, month=9, day=1)

START_TIME = datetime.datetime(hour=9, year=2024, month=10, day=1)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[datetime.timedelta(hours=1), datetime.timedelta(seconds=10)])



In [ ]:
# @title Generic memories are memories that all players and GM share.

shared_memories = [
    'There is a hamlet named Riverbend.',
    'Riverbend is an idyllic rural town.',
    'The river Solripple runs through the village of Riverbend.',
    'The Solripple is a mighty river.',
    'Riverbend has a temperate climate.',
    'Riverbend has a main street.',
    'There is a guitar store on Main street Riverbend.',
    'There is a grocery store on Main street Riverbend.',
    'There is a school on Main street Riverbend.',
    'There is a library on Main street Riverbend.',
    'Riverbend has only one pub.',
    'There is a pub on Main street Riverbend called The Sundrop Saloon.',
    'Town hall meetings often take place at The Sundrop Saloon.',
    'Riverbend does not have a park',
    'The main crop grown on the farms near Riverbend is alfalfa.',
    'Farms near Riverbend depend on water from the Solripple river.',
]

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)
print(shared_context)


## Functions to build the players

In [ ]:
# @title setup formative memory factories
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)
formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [ ]:
def build_agent(agent_config):

  mem = formative_memory_factory.make_memories(agent_config)

  # Build the player.

  time = components.report_state.ReportState(
    name='current_time',
    get_state=clock.current_time_interval_str)

  identity = components.identity.SimIdentity(model, mem, agent_config.name)
  goal_component = components.constant.ConstantConstruct(state=agent_config.goal)
  reflection = components.reflection.Reflection(
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      importance_threshold=15.0,
      verbose=False,
  )
  plan = components.plan.SimPlan(
      model,
      mem,
      agent_config.name,
      components=[identity, time],
      goal=goal_component,
      verbose=False,
  )
  current_obs = components.observation.Observation(agent_config.name, memory=mem)
  summary_obs = components.observation.ObservationSummary(
      model=model,
      agent_name=agent_config.name,
      components=[identity],
  )
  agent = basic_agent.BasicAgent(
      model,
      mem,
      agent_name=agent_config.name,
      clock=clock,
      verbose=True,
      components=[identity, plan, reflection, time, summary_obs, current_obs],
  )

  return agent

## Configure and build the players

In [ ]:
def make_random_big_five()->str:
  return str({
      'extraversion': random.randint(1, 10),
      'neuroticism': random.randint(1, 10),
      'openness': random.randint(1, 10),
      'conscientiousness': random.randint(1, 10),
      'agreeableness': random.randint(1, 10),
  })

scenario_premise = [

    (
        'Alice, Bob, Charlie and Dorothy are at the Sundrop Saloon. There '
        + 'is a snow storm and they have to wait it out inside.'
    ),
]
player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Organise a street party in Riverbend.',
        context=shared_context+'Alice is very socially active and knows everyone in town',
        traits = make_random_big_five()
            ),
    formative_memories.AgentConfig(
        name='Bob',
        gender='male',
        goal='Start a chess club in Riverbend.',
        context=shared_context + 'Bob is a chess enthusiast',
        traits = make_random_big_five()
            ),
    formative_memories.AgentConfig(
        name='Charlie',
        gender='male',
        goal='Organise an ale festival at the Sundrop Saloon.',
        context=shared_context + 'Charlie works at the Sundrop Saloon and loves real ales',
        traits = make_random_big_five()
            ),
    formative_memories.AgentConfig(
        name='Dorothy',
        gender='female',
        goal=(
            'Take students on a tour of Riverbend'
            ' it is funny.'
        ),
        context=shared_context + 'Dorothy is a teacher at school in Riverbend',
        traits = make_random_big_five()
            ),
    formative_memories.AgentConfig(
        name='Ellen',
        gender='female',
        goal=(
            'Write a paper on the history of Riverbend.'
        ),
        context=shared_context + 'Ellen is a librarian in the library in Riverbend',
        traits = make_random_big_five()
    ),
]

In [ ]:
NUM_PLAYERS = 3

player_configs = player_configs[:NUM_PLAYERS]
player_goals = {player_config.name: player_config.goal for player_config in player_configs}
players = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent in pool.map(build_agent, player_configs[:NUM_PLAYERS]):
    players.append(agent)

## Build GM

In [ ]:
game_master_instructions = (
    'This is a social science experiment. It is structured as a '
    'tabletop roleplaying game (like dungeons and dragons). You are the '
    'game master. You will describe the current situation to the '
    'participants in the experiment and then on the basis of what you '
    'tell them they will suggest actions for the character they control. '
    'Aside from you, each other participant controls just one character. '
    'You are the game master so you may control any non-player '
    'character. You will track the state of the world and keep it '
    'consistent as time passes in the simulation and the participants '
    'take actions and change things in their world. Remember that this '
    'is a serious social science experiment. It is not just a game. It '
    'need not be fun for the participants. Always use third-person '
    'limited perspective, even when speaking directly to the participants.'
)

In [ ]:
game_master_memory = associative_memory.AssociativeMemory(
    embedder, importance_model_gm.importance, clock=clock.now)

In [ ]:
# @title Create components and externalities
citizen_names = [player.name for player in players]
player_names = [player.name for player in players]

instructions_construct = components.constant.ConstantConstruct(
    game_master_instructions, 'Instructions'
)
facts_on_village = components.constant.ConstantConstruct(
    ' '.join(shared_memories), 'General knowledge of Riverbend'
)
player_status = gm_components.player_status.PlayerStatus(
    clock.now, model, game_master_memory, player_names
)

relevant_events = gm_components.relevant_events.RelevantEvents(
    clock.now, model, game_master_memory
)
time_display = gm_components.time_display.TimeDisplay(clock)


convo_externality = gm_components.conversation.Conversation(
    players,
    model,
    clock=clock,
    memory=game_master_memory,
    burner_memory_factory=blank_memory_factory,
    components=[player_status],
    cap_nonplayer_characters=2,
    game_master_instructions=game_master_instructions,
    shared_context=shared_context,
    verbose=False,
)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players,
    model=model,
    memory=game_master_memory,
    clock_now=clock.now,
    verbose=False,
    components=[player_status],
)

In [ ]:
# @title Metrics
goal_metric = goal_achievement.GoalAchievementMetric(model, player_goals, clock, 'Goal achievement', verbose=False)

metrics = [goal_metric]

In [ ]:
# @title Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    components=[
        instructions_construct,
        facts_on_village,
        player_status,
        convo_externality,
        direct_effect_externality,
        relevant_events,
        time_display,
    ],
    measurements=metrics,
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
)

## The RUN

In [ ]:
clock.set(START_TIME)

In [ ]:
#@title Initial observations and player location
for player in players:
  player.observe(
      f'{player.name} is at home, they have just woken up.'
  )
  game_master_memory.add(f'{player.name} is at their private home.')


In [ ]:
# @title Expect about 2-3 minutes per step.
episode_length = 12  # @param {type: 'integer'}
for _ in range(episode_length):
  env.step()


## Summary and analysis of the episode

In [ ]:
# @title Metrics plotting
tb = widgets.TabBar([metric.name() for metric in metrics])

for metric in metrics:
  with tb.output_to(metric.name()):
    plotting.plot_metric_line(metric)
    plotting.plot_metric_pie(metric)


In [ ]:
# @title Summarize the entire story.
all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

detailed_story = '\n'.join(all_gm_memories)
print('len(detailed_story): ', len(detailed_story))
# print(detailed_story)

episode_summary = model.sample_text(
    f'Sequence of events:\n{detailed_story}'+
    '\nNarratively summarize the above temporally ordered ' +
    'sequence of events. Write it as a news report. Summary:\n',
    max_characters=8000, max_tokens=8000, terminators=())
print(episode_summary)

In [ ]:
# @title Summarise the perspective of each player
player_logs = []
player_log_names = []
for player in players:
  name = player.name
  detailed_story = '\n'.join(player._memory.retrieve_recent(k=1000, add_time=True))
  summary = ''
  summary = model.sample_text(
      f'Sequence of events that happened to {name}:\n{detailed_story}'
      '\nWrite a short story that summarises these events.\n'
      ,
      max_characters=8000, max_tokens=8000, terminators=())

  all_player_mem = player._memory.retrieve_recent(k=1000, add_time=True)
  all_player_mem = ['Summary:', summary, 'Memories:'] + all_player_mem
  player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
  player_logs.append(player_html)
  player_log_names.append(f'{name}')


#Build and display HTML log of the experiment

In [ ]:
history_sources = [env, direct_effect_externality, convo_externality]
histories_html = [html_lib.PythonObjectToHTMLConverter(history.get_history()).convert() for history in history_sources]
histories_names = [history.name() for history in history_sources]

In [ ]:
gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

tabbed_html = html_lib.combine_html_pages(
    histories_html + [gm_mem_html] + player_logs,
    histories_names + ['GM'] + player_log_names,
    summary=episode_summary,
    title='Riverbend elections experiment',
)

tabbed_html = html_lib.finalise_html(tabbed_html)

In [ ]:
display.HTML(tabbed_html)

#Interact with a specific player

In [ ]:
sim_to_interact = 'Alice'  # @param ['Alice', 'Bob','Charlie', 'Dorothy', 'Ellen'] {type:"string"}
user_identity = 'a close friend'  # @param {type:"string"}
interaction_premise = f'{sim_to_interact} is talking to {user_identity}\n'  # @param {type:"string"}

player_names = [player.name for player in players]
player_by_name = {player.name: player for player in players}
selected_player = player_by_name[sim_to_interact]
interrogation = interaction_premise

In [ ]:
utterence_from_user = 'Did you win the elections?'  # @param {type:"string"}

interrogation += f'{user_identity}: {utterence_from_user}'
player_says = selected_player.say(interrogation)
interrogation += f'\n{sim_to_interact}: {player_says}\n'
print(interrogation)